# buy-and-hold

buy, then never ever sell, until the end date :)

In [1]:
# use future imports for python 3.x forward compatibility
from __future__ import print_function
from __future__ import unicode_literals
from __future__ import division
from __future__ import absolute_import

# other imports
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from talib.abstract import *

# project imports
import pinkfish as pf

# format price data
pd.options.display.float_format = '{:0.2f}'.format

%matplotlib inline

In [2]:
# set size of inline plots
'''note: rcParams can't be in same cell as import matplotlib
   or %matplotlib inline
   
   %matplotlib notebook: will lead to interactive plots embedded within
   the notebook, you can zoom and resize the figure
   
   %matplotlib inline: only draw static images in the notebook
'''
plt.rcParams["figure.figsize"] = (10, 7)

Some global data

In [3]:
symbol = '^GSPC'
#symbol = 'SPY'
capital = 10000
start = datetime.datetime(1900, 1, 1)
end = datetime.datetime.now()

Define Strategy Class

In [4]:
class Strategy(object):

    def __init__(self, symbol, capital, start, end):
        self._symbol = symbol
        self._capital = capital
        self._start = start
        self._end = end

    def _algo(self):
        self._tlog.cash = self._capital
        start_flag = True
        end_flag = False

        for i, row in enumerate(self._ts.itertuples()):

            date = row.Index.to_pydatetime()
            high = row.high
            low = row.low
            close = row.close
            end_flag = True if (i == len(self._ts) - 1) else False
            trade_state = None
            shares = 0

            if date < self._start:
                continue
            elif start_flag:
                start_flag = False
                # set start and end
                self._start = date
                self._end = self._ts.index[-1]

            # buy
            if self._tlog.num_open_trades() == 0:
                # enter buy in trade log
                shares = self._tlog.enter_trade(date, close)
            # sell
            elif end_flag:
                # enter sell in trade log
                shares = self._tlog.exit_trade(date, close)

            if shares > 0:
                trade_state = pf.TradeState.OPEN
                print("{0} BUY  {1} {2} @ {3:.2f}".format(
                      date, shares, self._symbol, close))
            elif shares < 0:
                trade_state = pf.TradeState.CLOSE
                print("{0} SELL {1} {2} @ {3:.2f}".format(
                      date, shares, self._symbol, close))
            else:
                trade_state = pf.TradeState.HOLD

            # record daily balance
            self._dbal.append(date, high, low, close,
                              self._tlog.shares, self._tlog.cash,
                              trade_state)

    def run(self):
        self._ts = pf.fetch_timeseries(self._symbol)
        self._ts = pf.select_tradeperiod(self._ts, self._start, self._end,
                                         use_adj=True, pad=False)
        self._tlog = pf.TradeLog()
        self._dbal = pf.DailyBal()

        self._algo()

    def get_logs(self):
        """ return DataFrames """
        tlog = self._tlog.get_log()
        dbal = self._dbal.get_log()
        return tlog, dbal

    def stats(self):
        tlog, dbal = self.get_logs()

        stats = pf.stats(self._ts, tlog, dbal,
                         self._start, self._end, self._capital)
        return stats

Run Strategy

In [5]:
s = Strategy(symbol, capital, start, end)
s.run()

1950-01-03 00:00:00 BUY  600 ^GSPC @ 16.66
2019-07-24 00:00:00 BUY  600 ^GSPC @ 3019.56


Retrieve log DataFrames

In [6]:
s.tlog, s.dbal = s.get_logs()
s.stats = s.stats()

In [7]:
s.tlog.tail()

,entry_date,entry_price,exit_date,exit_price,pl_points,pl_cash,qty,cumul_total
0,1950-01-03,16.66,2019-07-24,3019.56,3002.90,1801740.04,600,1801740.04


In [8]:
s.dbal.tail()

,high,low,close,shares,cash,state
date,,,,,,
2019-07-18,1798972.02,1783858.05,1797070.06,600,4.00,1
2019-07-19,1803616.01,1785520.06,1785970.06,600,4.00,1
2019-07-22,1794429.98,1785993.94,1791022.02,600,4.00,1
2019-07-23,1803543.94,1793140.04,1803285.98,600,4.00,1
2019-07-24,1811740.04,1811740.04,1811740.04,0,1811740.04,0


In [9]:
pf.print_full(s.stats)

start                                                   1950-01-03
end                                                     2019-07-24
beginning_balance                                            10000
ending_balance                                          1811740.04
total_net_profit                                        1801740.04
gross_profit                                            1801740.04
gross_loss                                                    0.00
profit_factor                                                 1000
return_on_initial_capital                                 18017.40
annual_return_rate                                            7.76
trading_period                           69 years 6 months 21 days
pct_time_in_market                                          100.00
total_num_trades                                                 1
num_winning_trades                                               1
num_losing_trades                                             

Summary

In [10]:
metrics = ('annual_return_rate',
           'max_closed_out_drawdown',
           'drawdown_annualized_return',
           'drawdown_recovery',
           'best_month',
           'worst_month',
           'sharpe_ratio',
           'sortino_ratio',
           'monthly_std')

pf.summary(s.stats, *metrics)

,strategy
annual_return_rate,7.76
max_closed_out_drawdown,-56.78
drawdown_annualized_return,-7.31
drawdown_recovery,-1.42
best_month,23.49
worst_month,-29.56
sharpe_ratio,0.57
sortino_ratio,0.73
monthly_std,4.14


In [11]:
returns = s.dbal['close']
pf.monthly_returns_map(returns['1990':])

Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Year
1990,-8.5,0.9,2.4,-2.7,9.2,-0.9,-0.5,-9.4,-5.1,-0.7,6.0,2.5,-8.2
1991,4.2,6.7,2.2,0.0,3.9,-4.8,4.5,2.0,-1.9,1.2,-4.4,11.2,26.3
1992,-2.0,1.0,-2.2,2.8,0.1,-1.7,3.9,-2.4,0.9,0.2,3.0,1.0,4.5
1993,0.7,1.0,1.9,-2.5,2.3,0.1,-0.5,3.4,-1.0,1.9,-1.3,1.0,7.1
1994,3.3,-3.0,-4.6,1.2,1.2,-2.7,3.1,3.8,-2.7,2.1,-4.0,1.2,-1.5
1995,2.4,3.6,2.7,2.8,3.6,2.1,3.2,-0.0,4.0,-0.5,4.1,1.7,34.1
1996,3.3,0.7,0.8,1.3,2.3,0.2,-4.6,1.9,5.4,2.6,7.3,-2.2,20.3
1997,6.1,0.6,-4.3,5.8,5.9,4.3,7.8,-5.7,5.3,-3.4,4.5,1.6,31.0
1998,1.0,7.0,5.0,0.9,-1.9,3.9,-1.2,-14.6,6.2,8.0,5.9,5.6,26.7
1999,4.1,-3.2,3.9,3.8,-2.5,5.4,-3.2,-0.6,-2.9,6.3,1.9,5.8,19.5


In [12]:
returns = s.dbal['close']
pf.holding_period_map(returns['1990':])